### Import all the relevant libraries

In [ ]:
import itertools
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import sobol_seq

### Define a few functions to be used in the computation

In [ ]:
def Y(Y0,Beta,Epsilon,r,N0,Gamma,Year0):
    return Y0+10**(a+b*Beta+Epsilon)*Beta*r*((N0*1000)**(1-Gamma)+r*(2050-Year0)*(1-Gamma))**((Beta+Gamma-1)/
    (1-Gamma))

In [ ]:
def create_dict(key, values):
    return dict(zip(key, values))

In [ ]:
def trigger_index(t):
    return np.sum(t*[len(lu)/len(Areas2),len(lu)/(len(Areas2)*len(dataSets)),len(lu)/(len(Areas2)*len(dataSets)*
len(Regression_Method)),len(lu)/(len(Areas2)*len(dataSets)*len(Regression_Method)*len(Robustness)),1],axis=1).astype(int)

### Read parameters

In [ ]:
p3 = pd.read_csv('sample.matrix.csv')

### Prepare the dataset

In [ ]:
p3 = p3.replace({'Continent':{1:'Africa',2:'Americas',3:'Asia',4:'Europe'},
'X1':{1:'Aquastat',2:'FAOSTAT',3:'Siebert.et.al.2013',4:'Meier.et.al.2018',5:'Salmon.et.al.2015',6:'Thenkabail.et.al.2009'},
'X2':{1:'OLS',2:'SMA'},'X3':{1:'NO',2:'YES'}})
p3 = p3.rename(index=str, columns={"X1":"Dataset","X2":"Regression","X3":"Robust"})

### Set the lookup table

In [ ]:
lu = pd.read_csv('lookup.csv')

In [ ]:
lu = lu.replace({'Continent':{1:'Africa',2:'Americas',3:'Asia',4:'Europe'},
'Dataset':{1:'Aquastat',2:'FAOSTAT',3:'Siebert.et.al.2013',4:'Meier.et.al.2018',5:'Salmon.et.al.2015',6:'Thenkabail.et.al.2009'},
'Regression':{1:'OLS',2:'SMA'},'Robust':{1:'NO',2:'YES'}})
Continent = lu['Continent']
lu.drop(labels=['Continent'], axis=1,inplace = True)
lu.insert(0, 'Continent', Continent)
lu = lu.sort_values(by=['Continent','Dataset','Regression','Robust','Beta'])
lu = lu.set_index(['Continent','Dataset','Regression','Robust']).sort_index()

In [ ]:
lu = lu.reset_index()

### Set the irrigated area - Y0 in our simulation

In [ ]:
irrArea = pd.read_excel('full.dataset2.xlsx',\
                      sheet_name='meier')

Areas2 = ['Africa','Americas','Asia','Europe']
dataSets = ['Aquastat','FAOSTAT','Meier.et.al.2018','Salmon.et.al.2015','Siebert.et.al.2013','Thenkabail.et.al.2009']
Regression_Method = ['OLS','SMA']
Robustness = ['YES','NO']
sl = [['OLS','NO'],['OLS','YES'],['SMA','NO'],['SMA','YES']]
Y0 = pd.DataFrame([irrArea[irrArea['Continent']==ar2].sum() for ar2 in Areas2], index=[ar2 for ar2 in Areas2])
Y0=Y0.iloc[:,3:]
Y0=Y0.reindex(sorted(Y0.columns), axis=1)

### The K setting the carrying capacity factor

In [ ]:
K = pd.DataFrame([[7.14e8,8.85e8,6.68e8,4.78e8],[1.205e9,1.481e9,8.89e8,7.05e8]],columns=[ar2 for ar2 in Areas2],index=\
                    ['Net','Gross']).T

### Outliers count

In [ ]:
outliers = pd.read_csv('out.df.csv').set_index(['Continent','Dataset'])

### Quasi-random matrix for the Monte Carlo simulations

In [ ]:
qM =sobol_seq.i4_sobol_generate(21,100000)

### Building the triggers for the lookup table

In [ ]:
triggers = np.vstack((np.array(list(itertools.chain.from_iterable(itertools.repeat(x, 100000) 
for x in range(len(Areas2))))),np.tile((qM[:100000,0:4]*[6,2,1,20000]).T,4)))

triggersB = np.vstack((np.array(list(itertools.chain.from_iterable(itertools.repeat(x, 100000) 
for x in range(len(Areas2))))),np.tile((qM[:100000,10:14]*[6,2,1,20000]).T,4)))

In [ ]:
la = np.array([0 if t2[3] > 1/(2+outliers['Outliers'][ar2,dataSets[t2[1].astype(int)]]) else 1 for ar2 in Areas2 
               for t2 in triggers[:,:100000].T])
lb = np.array([0 if t2b[3] > 1/(2+outliers['Outliers'][ar2,dataSets[t2b[1].astype(int)]]) else 1 for ar2 in Areas2 
               for t2b in triggersB[:,:100000].T])

In [ ]:
triggers[3]=la
triggersB[3]=lb
triggers = triggers.astype(int)
triggersB = triggersB.astype(int)

### Adjust the database to 100k values

In [ ]:
p4 = pd.concat([pd.Series(p3.r[p3.Continent==ar2].iloc[:int(len(p3[p3.Continent==ar2])/2)]) for ar2 in Areas2])
p4B = pd.concat([pd.Series(p3.r[p3.Continent==ar2].iloc[int(len(p3[p3.Continent==ar2])/2):]) for ar2 in Areas2])

### Define the coefficients for the alpha regression in terms of Beta

In [ ]:
a = 5.51242451040592
b = -7.24661215514772
epsilon = 0.157964920922199

### Population initial figures

In [ ]:
PPop = pd.read_csv('UN_PP1999-2012.csv')
del PPop['Variant']
PPop = PPop.rename(columns={'Country or Area':'Continent','Year(s)':'Year'})
PPop=PPop.pivot(index='Continent',columns='Year',values='Value')
PPop.loc['Americas']=PPop.loc['Latin America and the Caribbean']+PPop.loc['Northern America']
PPop = PPop.drop(['Latin America and the Caribbean','Northern America'])
PPop = PPop.sort_index()

In [ ]:
tr_As = [triggers] 
for tr in range(1,len(triggers)):
    tr_As.append(triggers.copy())
    tr_As[tr][tr]=triggersB[tr]
    
tr_Bs = [triggersB] 
for trB in range(1,len(triggers)):
    tr_Bs.append(triggersB.copy())
    tr_Bs[trB][trB]=triggers[trB]
    
tIndexA = [trigger_index(tri.T) for tri in tr_As]
tIndexB = [trigger_index(triB.T) for triB in tr_Bs]

In [ ]:
sM_As = []
for itI,tI in enumerate(tIndexA):
    sM_As.append(lu.copy())
    sM_As[itI]=sM_As[itI].reindex(tI)
    sM_As[itI]=sM_As[itI].reset_index(drop=True)
    
sM_Bs = []
for itIB,tIB in enumerate(tIndexB):
    sM_Bs.append(lu.copy())
    sM_Bs[itIB]=sM_Bs[itIB].reindex(tIB)
    sM_Bs[itIB]=sM_Bs[itIB].reset_index(drop=True)

### Fix all the parameters

In [ ]:
for sm in sM_As:
    sm['r']=p4.values
    sm['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,6]-1),4)
    sm['Year0'] = np.tile((PPop.columns.min()+(PPop.columns.max()-PPop.columns.min()+1)*qM[:,7]).astype(int),4)
    sm['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,8]-1)+1,4)
    sm['Y0']=0
    sm['K']=0
    for ar2 in Areas2:
        sm['Y0'][sm['Continent']==ar2]=Y0.loc[ar2].min()+qM[:,5]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        sm['K'][sm['Continent']==ar2]=K['Net'].loc[ar2]+qM[:,9]*(K['Gross'].loc[ar2]-K['Net'].loc[ar2])
    sm['N0']=[PPop[smy][ar2] for ar2 in Areas2 for smy in sm['Year0'][:int(len(sm)/4)]]
        
for smB in sM_Bs:
    smB['r']=p4B.values
    smB['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,16]-1),4)
    smB['Year0'] = np.tile((PPop.columns.min()+(PPop.columns.max()-PPop.columns.min()+1)*qM[:,17]).astype(int),4)
    smB['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,18]-1)+1,4)
    smB['Y0']=0
    smB['K']=0
    for ar2 in Areas2:
        smB['Y0'][smB['Continent']==ar2]=Y0.loc[ar2].min()+qM[:,15]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        smB['K'][smB['Continent']==ar2]=K['Net'].loc[ar2]+qM[:,19]*(K['Gross'].loc[ar2]-K['Net'].loc[ar2])
    smB['N0']=[PPop[smy][ar2] for ar2 in Areas2 for smy in smB['Year0'][:int(len(smB)/4)]]

### Define the variables list for the scrambled matrices

In [ ]:
variables2 = [['r'],['Y0'],['Epsilon'],['Year0','N0'],['Gamma'],['K']]

In [ ]:
variables = [['Dataset'],['Regression'],['Robust'],['X4'],['r'],['Y0'],['Epsilon'],['Year0','N0'],['Gamma'],['K']]

In [ ]:
for iv,v in enumerate(variables2):
    sM_As.append(sM_As[0].copy())
    sM_As[-1][v]=sM_Bs[0][v]
    
    sM_Bs.append(sM_Bs[0].copy())
    sM_Bs[-1][v]=sM_As[0][v]

### Calculate Y and replace overshooting Y values with K figures

In [ ]:
for sm in sM_As:
    sm['Y']=Y(sm.Y0,sm.Beta,sm.Epsilon,sm.r,sm.N0,sm.Gamma,sm.Year0)
    sm['Y'].loc[sm['Y']>sm.K] = sm.K

for smB in sM_Bs:
    smB['Y']=Y(smB.Y0,smB.Beta,smB.Epsilon,smB.r,smB.N0,smB.Gamma,smB.Year0)
    smB['Y'].loc[smB['Y']>smB.K] = smB.K

### Filter out negative values

In [ ]:
sM_As = [sm[sm['Y']>0] for sm in sM_As]

sM_Bs = [smB[smB['Y']>0] for smB in sM_Bs]

### Append the B matrix for the Sobol-indices computation

In [ ]:
sM_S = sM_As.copy()
sM_S.append(sM_Bs[0])

### Narrow down the indices in order to get rid of potential 'orphan' (uncoupled) rows across the matrices

In [ ]:
inr = sM_S[0].index.intersection(sM_S[1].index).intersection(sM_S[2].index).intersection(sM_S[3].index).\
intersection(sM_S[4].index).intersection(sM_S[5].index).intersection(sM_S[6].index).intersection(sM_S[7].index).\
intersection(sM_S[8].index).intersection(sM_S[9].index).intersection(sM_S[10].index).intersection(sM_S[11].index)

sM_S = [sms.merge(pd.DataFrame(index=inr), left_index=True, right_index=True, how='right') for sms in sM_S]

### Assess the Global Irrigated Area in 2050

In [ ]:
worldIrrigatedArea = pd.concat([sM_As[0].Y[ia*int(len(sM_As[0].Y)/len(Areas2)):(ia+1)*int(len(sM_As[0].Y)/len(Areas2))].reset_index(drop=True) 
                                for ia in range(len(Areas2))],axis=1,ignore_index=True).sum(axis=1)

### Describe the statistical properties of the datasets used for the four continents

In [ ]:
# Seed for bootstrapping
qMbS = sobol_seq.i4_sobol_generate(1,len(sM_As[0])*1000)

### Bootstrap sensitivity indices

In [ ]:
%%time
c_r = []
T_r = []
Sa_r = []
for r in range(1000):    
    seed = (qMbS[r*len(sM_As[0]):(r+1)*len(sM_As[0]),0]*len(sM_As[0])).astype(int)
    
    sM_BS = [smS.merge(pd.DataFrame(index=seed), left_index=True, right_index=True, how='right') for smS in sM_S]

    V = []
    Sa_V = []
    T_V = []
    for imb,mb in enumerate(sM_BS[1:-1]):
        Geo = []
        Sa_G = []
        T_G = []
        for ar2 in Areas2:
            Sa_G.append(((mb['Y'][mb['Continent']==ar2]-sM_BS[0]['Y'][sM_BS[0]['Continent']==ar2])* sM_BS[-1]['Y'][sM_BS[-1]['Continent']==ar2]).mean()/\
            pd.concat([sM_BS[-1]['Y'][sM_BS[-1]['Continent']==ar2],sM_BS[0]['Y'][sM_BS[0]['Continent']==ar2]]).var(ddof=0))
            T_G.append(0.5*((mb['Y'][mb['Continent']==ar2]-sM_BS[0]['Y'][sM_BS[0]['Continent']==ar2])**2).mean()/\
            pd.concat([sM_BS[-1]['Y'][sM_BS[-1]['Continent']==ar2],sM_BS[0]['Y'][sM_BS[0]['Continent']==ar2]]).var(ddof=0))
            Geo.append(ar2)
        Sa_G_dic = create_dict(Geo,Sa_G)
        T_G_dic = create_dict(Geo,T_G)
        V.append(variables[imb][0])
        Sa_V.append(Sa_G_dic)
        T_V.append(T_G_dic)
    Sa_V_dic = create_dict(V,Sa_V)
    T_V_dic = create_dict(V,T_V)
    
    c_r.append(r)
    Sa_r.append(Sa_V_dic)
    T_r.append(T_V_dic)
Sa_r_dic = create_dict(c_r,Sa_r)
T_r_dic = create_dict(c_r,T_r)
Sa_of_df = {Sa_k: pd.DataFrame(Sa_v) for Sa_k,Sa_v in Sa_r_dic.items()}
T_of_df = {T_k: pd.DataFrame(T_v) for T_k,T_v in T_r_dic.items()}
T_df = pd.concat(T_of_df, axis=0)
Sa_df = pd.concat(Sa_of_df, axis=0)

### Irrigated Land Area Distribution plots

In [ ]:
co = ['b','r','g','k','c','m']
plt.style.use('ggplot')
for ar2 in Areas2:
    ax = sns.distplot(sM_As[0].Y[sM_As[0].Continent==ar2], bins=500, kde=False, axlabel='Irrigated Area 2050 (ha)_'+str(ar2))
    ax.set_xscale('log')
    ax.set_ylabel('Count')
    for ds in K:
        plt.axvline(x=K[ds][ar2],label=str(ds)+' Cropland in 2050')
    plt.legend()
    plt.savefig('Uncertainty Analysis_'+str(ar2))
    plt.close()

### Distribution plots World Area

In [ ]:
plt.style.use('ggplot')
ax = sns.distplot(worldIrrigatedArea, bins=200, kde=False, axlabel='Global Irrigated Area 2050 (ha)')
ax.set_xscale('log')
ax.set_ylabel('Count')
for ds in Y0:
    plt.plot([Y0[ds].sum(),Y0[ds].sum()], [0,16000], '--', linewidth=2, label=ds)
plt.yticks(np.arange(0, 2e4, step=5e3))
plt.legend()
plt.savefig('Uncertainty Analysis_World')
plt.close()

### Scatter Plots

In [ ]:
co = ['Dataset','Regression','Robust','r','Y0','Epsilon','Year0','Gamma','K']
for ar2 in Areas2:
    for col in co:
        plt.figure(figsize=(20,10))
        plt.scatter(sM_As[0][col][sM_As[0].Continent==ar2],sM_As[0]['Y'][sM_As[0].Continent==ar2],s=1,label=str(ar2)+'_'+str(col))
        plt.yscale('log')
        plt.ylim(1e6,2e9)
        plt.legend()
        plt.savefig('ScatterPlot_'+str(col)+str(ar2))
        plt.close()

### Sensitivity-indices boxplots

In [ ]:
for ar2 in Areas2:
    fig, ax = plt.subplots(figsize=(20,10))
    for iv,v in enumerate(variables):
        T_df.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='T', color='b',patch_artist=True)
        Sa_df.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='S',positions=[iv+0.5 for iv in range(len(variables))],
                                              color='r',patch_artist=True)
    m_patch = mpatches.Patch(color='b', label='T')
    c_patch = mpatches.Patch(color='r', label='S')
    plt.legend(handles=[m_patch,c_patch])
    plt.xticks([iv+0.75 for iv in range(len(variables))])
    plt.xlim(0,10.5)
    plt.title(ar2)
    plt.savefig('Confidence_Interval_Sobol_Indices_'+str(ar2))
    plt.close()

### Defining variables for the sensitivity analysis of the cluster

In [ ]:
variables3 = [['Dataset','Y0'],['Regression','Robust','X4','Epsilon'],['r','Gamma']]

In [ ]:
tr_As2 = [triggers] 
for tr2 in range(1,len(variables3)):
    tr_As2.append(triggers.copy())
    
tr_Bs2 = [triggersB] 
for trB2 in range(1,len(variables3)):
    tr_Bs2.append(triggersB.copy())

tr_As2[1][1:4]=tr_Bs2[0][1:4]
tr_As2[-1][-1]=tr_Bs2[0][-1]

tr_Bs2[1][1:4]=tr_As2[0][1:4]
tr_Bs2[-1][-1]=tr_As2[0][-1]

tIndexA2 = [trigger_index(tri2.T) for tri2 in tr_As2]
tIndexB2 = [trigger_index(triB2.T) for triB2 in tr_Bs2]

In [ ]:
sM_As2 = []
for itI2,tI2 in enumerate(tIndexA2):
    sM_As2.append(lu.copy())
    sM_As2[itI2]=sM_As2[itI2].reindex(tI2)
    sM_As2[itI2]=sM_As2[itI2].reset_index(drop=True)
    
sM_Bs2 = []
for itIB2,tIB2 in enumerate(tIndexB2):
    sM_Bs2.append(lu.copy())
    sM_Bs2[itIB2]=sM_Bs2[itIB2].reindex(tIB2)
    sM_Bs2[itIB2]=sM_Bs2[itIB2].reset_index(drop=True)

In [ ]:
for sm2 in sM_As2:
    sm2['r']=p4.values
    sm2['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,6]-1),4)
    sm2['Year0'] = np.mean(PPop.columns).astype(int)
    sm2['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,8]-1)+1,4)
    sm2['Y0']=0
    sm2['K']=0
    sm2['N0']=0
    for ar2 in Areas2:
        sm2['Y0'][sm2['Continent']==ar2]=Y0.loc[ar2].min()+qM[:,5]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        sm2['K'][sm2['Continent']==ar2]=np.mean([K['Net'].loc[ar2],K['Gross'].loc[ar2]])
        sm2['N0'][sm2['Continent']==ar2]=PPop[np.mean(PPop.columns).astype(int)].loc[ar2]
        
for smB2 in sM_Bs2:
    smB2['r']=p4B.values
    smB2['Epsilon']=np.tile(epsilon*2**0.5*sp.special.erfinv(2*qM[:,16]-1),4)
    smB2['Year0'] = np.mean(PPop.columns).astype(int)
    smB2['Gamma']= np.tile(0.02*2**0.5*sp.special.erfinv(2*qM[:,18]-1)+1,4)
    smB2['Y0']=0
    smB2['K']=0
    smB2['N0']=0
    for ar2 in Areas2:
        smB2['Y0'][smB2['Continent']==ar2]=Y0.loc[ar2].min()+qM[:,15]*(Y0.loc[ar2].max()-Y0.loc[ar2].min())
        smB2['K'][smB2['Continent']==ar2]=np.mean([K['Net'].loc[ar2],K['Gross'].loc[ar2]])
        smB2['N0'][smB2['Continent']==ar2]=PPop[np.mean(PPop.columns).astype(int)].loc[ar2]

### Generate the scrambled matrices

In [ ]:
sM_As2.append(sM_As2[0].copy())
sM_Bs2.append(sM_Bs2[0].copy())

sM_As2[1]['Y0']=sM_Bs2[0]['Y0']
sM_As2[2]['Epsilon']=sM_Bs2[0]['Epsilon']
sM_As2[-1][['Gamma','r']]=sM_Bs2[0][['Gamma','r']]

### Calculate Y and replace overshooting Y values with K figures

In [ ]:
for sm2 in sM_As2:
    sm2['Y']=Y(sm2.Y0,sm2.Beta,sm2.Epsilon,sm2.r,sm2.N0,sm2.Gamma,sm2.Year0)
    sm2['Y'].loc[sm2['Y']>sm2.K] = sm2.K
    
for smB2 in sM_Bs2:
    smB2['Y']=Y(smB2.Y0,smB2.Beta,smB2.Epsilon,smB2.r,smB2.N0,smB2.Gamma,smB2.Year0)
    smB2['Y'].loc[smB2['Y']>smB2.K] = smB2.K

### Filter out negative values

In [ ]:
sM_As2 = [sm2[sm2['Y']>0] for sm2 in sM_As2]

sM_Bs2 = [smB2[smB2['Y']>0] for smB2 in sM_Bs2]

In [ ]:
sM_S2 = sM_As2.copy()
sM_S2.append(sM_Bs2[0])

In [ ]:
inr2 = sM_S2[0].index.intersection(sM_S2[1].index).intersection(sM_S2[2].index).intersection(sM_S2[3].index).\
intersection(sM_S2[4].index)

sM_S2 = [sms2.merge(pd.DataFrame(index=inr2), left_index=True, right_index=True, how='right') for sms2 in sM_S2]

### Append B matrix for the Sobol indices

In [ ]:
# Seed for bootstrapping
qMbS = sobol_seq.i4_sobol_generate(1,len(sM_S2[0])*1000)

In [ ]:
c_r2 = []
T_r2 = []
Sa_r2 = []
for r in range(1000):    
    seed = (qMbS[r*len(sM_As2[0]):(r+1)*len(sM_As2[0]),0]*len(sM_As2[0])).astype(int)
    
    sM_BS2 = [sms2.merge(pd.DataFrame(index=seed), left_index=True, right_index=True, how='right') for sms2 in sM_S2]

    V = []
    Sa_V = []
    T_V = []
    for imb,mb in enumerate(sM_BS2[1:-1]):
        Geo = []
        Sa_G = []
        T_G = []
        for ar2 in Areas2:
            Sa_G.append(((mb['Y'][mb['Continent']==ar2]-sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2])*
            sM_BS2[-1]['Y'][sM_BS2[-1]['Continent']==ar2]).mean()/\
            pd.concat([sM_BS2[-1]['Y'][sM_BS2[-1]['Continent']==ar2],sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2]]).var(ddof=0))
            T_G.append(0.5*((mb['Y'][mb['Continent']==ar2]-sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2])**2).mean()/\
            pd.concat([sM_BS2[-1]['Y'][sM_BS2[-1]['Continent']==ar2],sM_BS2[0]['Y'][sM_BS2[0]['Continent']==ar2]]).var(ddof=0))
            Geo.append(ar2)
        Sa_G_dic = create_dict(Geo,Sa_G)
        T_G_dic = create_dict(Geo,T_G)
        V.append(variables3[imb][0])
        Sa_V.append(Sa_G_dic)
        T_V.append(T_G_dic)
    Sa_V_dic = create_dict(V,Sa_V)
    T_V_dic = create_dict(V,T_V)
    
    c_r2.append(r)
    Sa_r2.append(Sa_V_dic)
    T_r2.append(T_V_dic)
    
Sa_r_dic2 = create_dict(c_r2,Sa_r2)
T_r_dic2 = create_dict(c_r2,T_r2)
Sa_of_df2 = {Sa_k2: pd.DataFrame(Sa_v2) for Sa_k2,Sa_v2 in Sa_r_dic2.items()}
T_of_df2 = {T_k2: pd.DataFrame(T_v2) for T_k2,T_v2 in T_r_dic2.items()}
T_df2 = pd.concat(T_of_df2, axis=0)
Sa_df2 = pd.concat(Sa_of_df2, axis=0)

In [ ]:
sOl = []
for cS in Sa_df2:
    sOl.append(1-Sa_df2.sum(axis=1)+Sa_df2[cS]-T_df2[cS])
secondOrder = pd.concat(sOl, axis=1).rename(columns={0:'S_Model-Population',1:'S_Irrigation-Population',
                                                     2:'S_Irrigation-Model'})
thirdOrder = T_df2.sum(axis=1)+Sa_df2.sum(axis=1)-2

SIs = pd.concat([Sa_df2,secondOrder,thirdOrder],axis=1).rename(columns={'Dataset':'S_Irrigation','Regression':'S_Model',
'r':'S_Population',0:'S_third_order'})

### Sensitivity indices boxplots

In [ ]:
for ar2 in Areas2:
    fig, ax = plt.subplots(figsize=(20,10))
    co = -1
    for cSs in SIs:
        co+=1
        SIs.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label=cSs,patch_artist=True)
    plt.xlim(0,7.5)
    plt.ylim(-0.1,0.8)
    plt.title(ar2)
    plt.savefig('Sensitivity_Indices_Clusters_'+str(ar2))
    plt.close()

In [83]:
variables4 = ['Irrigation','Model','Population']

In [88]:
T_df2 = T_df2.rename(columns={'Dataset':'Irrigation','Regression':'Model','r':'Population'})
Sa_df2 = Sa_df2.rename(columns={'Dataset':'Irrigation','Regression':'Model','r':'Population'})

In [90]:
for ar2 in Areas2:
    fig, ax = plt.subplots(figsize=(20,10))
    for iv,v in enumerate(variables4):
        T_df2.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='T', color='b',patch_artist=True)
        Sa_df2.loc[pd.IndexSlice[:, ar2],:].plot(kind='box',ax=ax,label='S',positions=[iv+0.5 for iv in range(len(variables4))],
                                              color='r',patch_artist=True)
    m_patch = mpatches.Patch(color='b', label='T')
    c_patch = mpatches.Patch(color='r', label='S')
    plt.legend(handles=[m_patch,c_patch])
    plt.xticks([iv+0.75 for iv in range(len(variables4))])
    plt.xlim(0,3.5)
    plt.title(ar2)
    plt.savefig('Confidence_Interval_Sobol_Indices_Clusters_'+str(ar2))
    plt.close()